In [1]:
import requests
import os
import uuid
import re
import datetime

In [2]:
base_url = 'http://localhost:8000/api/v0.0'
ngsi_msg = {
  "headers": [
    {
      "fiware-service": "tdm"
    },
    {
      "fiware-servicePath": "/cagliari/edge/meteo"
    },
    {
      "timestamp": 1531774294021
    }
  ],
  "body": {
    "attributes": [
      {
        "name": "altitude",
        "type": "float",
        "value": " "
      },
      {
        "name": "dateObserved",
        "type": "String",
        "value": "2018-07-16T20:51:33+00:00"
      },
      {
        "name": "location",
        "type": "geo:point",
        "value": "39.2479168, 9.1329701"
      },
      {
        "name": "timestamp",
        "type": "Integer",
        "value": "1531774293"
      },
      {
        "name": "windDirection",
        "type": "Float",
        "value": "174.545"
      },
      {
        "name": "windSpeed",
        "type": "Float",
        "value": "1.000"
      },
      {
        "name": "latitude",
        "type": "string",
        "value": "39.2479168"
      },
      {
        "name": "longitude",
        "type": "string",
        "value": "9.1329701"
      }
    ],
    "type": "WeatherObserved",
    "isPattern": "false",
    "id": "WeatherObserved:Edge-CFA703F4.esp8266-7806085.Davis"
  }
}

In [3]:
p = re.compile(r'(?P<Type>\w+):(?P<Edge>[a-zA-Z0-9_-]+)\.(?P<Node>[a-zA-Z0-9_-]+)\.(?P<Sensor>[a-zA-Z0-9_-]+)')
msg_type, node_name, station_name, sensor_type_name = p.search(ngsi_msg['body']['id']).groups()
msg_type, node_name, station_name, sensor_type_name

('WeatherObserved', 'Edge-CFA703F4', 'esp8266-7806085', 'Davis')

In [4]:
sensor_name = '{}.{}'.format(station_name, sensor_type_name)
sensor_name

'esp8266-7806085.Davis'

In [5]:
def get_geometry(attributes):
    geom = {}
    for attr in attributes:
        if attr['name'] in {'latitude', 'longitude'}:
            geom[attr['name']] = float(attr['value'])
    return [geom['longitude'], geom['latitude']]   

In [6]:
geometry = {"type": "Point", "coordinates": get_geometry(ngsi_msg['body']['attributes'])}
geometry

{'type': 'Point', 'coordinates': [9.1329701, 39.2479168]}

In [7]:
non_properties = {'latitude', 'longitude', 'timestamp', 'dateObserved', 'location'}
properties = [ attr['name'] for attr in ngsi_msg['body']['attributes'] if attr['name'] not in non_properties]
sensor_type = {
        "type": msg_type,
        "name": sensor_type_name,
        "category": ["sensor"],
        "function": ["sensing"],
        "controlledProperty": properties
}
sensor_type

{'type': 'WeatherObserved',
 'name': 'Davis',
 'category': ['sensor'],
 'function': ['sensing'],
 'controlledProperty': ['altitude', 'windDirection', 'windSpeed']}

In [8]:
codes = requests.post(os.path.join(base_url, 'sensor_types'), json=[sensor_type]).json()
codes

['a47d9dd5-9491-5e86-a908-ac698dc4cca0']

In [9]:
stypecode = codes[0]
stypecode

'a47d9dd5-9491-5e86-a908-ac698dc4cca0'

In [11]:
sensors = [{
    'name': f"{sensor_name}.{prop}",
    'type': sensor_type_name,
    'node': node_name,
    'geometry': geometry,
    'controlledProperty': prop
    } for prop in properties]
sensors

[{'name': 'esp8266-7806085.Davis.altitude',
  'type': 'Davis',
  'node': 'Edge-CFA703F4',
  'geometry': {'type': 'Point', 'coordinates': [9.1329701, 39.2479168]},
  'controlledProperty': 'altitude'},
 {'name': 'esp8266-7806085.Davis.windDirection',
  'type': 'Davis',
  'node': 'Edge-CFA703F4',
  'geometry': {'type': 'Point', 'coordinates': [9.1329701, 39.2479168]},
  'controlledProperty': 'windDirection'},
 {'name': 'esp8266-7806085.Davis.windSpeed',
  'type': 'Davis',
  'node': 'Edge-CFA703F4',
  'geometry': {'type': 'Point', 'coordinates': [9.1329701, 39.2479168]},
  'controlledProperty': 'windSpeed'}]

In [12]:
codes = requests.post(os.path.join(base_url, 'sensors'), json=sensors).json()
codes

['c71435dc-c644-5a19-932a-5737bdfdb5bd',
 '13a87b2f-9723-508e-8780-1dfdd8112f45',
 'd74b6f7e-6c99-5f32-89a4-6d060c134a3c']

In [15]:
non_properties

{'dateObserved', 'latitude', 'location', 'longitude', 'timestamp'}

In [20]:
measures = {}
time = None
to_skip = {'dateObserved', 'location', 'latitude', 'longitude'}
for attr in ngsi_msg['body']['attributes']:
    name = attr['name']
    value = attr['value']
    if value is not None and str(value).strip() and name not in to_skip:
        if name == 'timestamp':
            time = datetime.datetime.fromtimestamp(float(value))
        else:             
            measures[name] = float(value)
time, measures

(datetime.datetime(2018, 7, 16, 22, 51, 33),
 {'windDirection': 174.545, 'windSpeed': 1.0})

In [21]:
dtime = time.strftime('%Y-%m-%dT%H:%M:%SZ')
dtime

'2018-07-16T22:51:33Z'

In [23]:
json_measures = [
    {"time": dtime, "sensor": f"{sensor_name}.{p}", "measure": {"value": measures[p]}}
    for p in measures
]
json_measures

[{'time': '2018-07-16T22:51:33Z',
  'sensor': 'esp8266-7806085.Davis.windDirection',
  'measure': {'value': 174.545}},
 {'time': '2018-07-16T22:51:33Z',
  'sensor': 'esp8266-7806085.Davis.windSpeed',
  'measure': {'value': 1.0}}]

In [24]:
requests.post(os.path.join(base_url, 'measures'), json=json_measures).json()

{'loaded': 2}

In [25]:
args = {
    'after': '2018-01-01T00:00:00Z',
    'before':'2019-01-01T00:00:00Z'
}
for c in codes:
    print(requests.get(os.path.join(base_url, 'sensors', c,'timeseries'), params=args).json())

{'data': [], 'timebase': '2018-01-01T00:00:00Z', 'timedelta': []}
{'data': [174.545], 'timebase': '2018-01-01T00:00:00Z', 'timedelta': [17016693.0]}
{'data': [1.0], 'timebase': '2018-01-01T00:00:00Z', 'timedelta': [17016693.0]}
